<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.8 MB/s eta 0:00:00


In [3]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [5]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

from sklearn.model_selection import GridSearchCV

**Non-binary**

All features

In [9]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [10]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [16]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    MLPClassifier(),
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'random_state': [1, 2, 3],
        'solver': ['sgd', 'adam']
    },
    n_iter=32,
    cv=3
)

opt.fit(X_trainscaled, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_testscaled, y_test))

val. score: 0.9013666072489602
test score: 0.8823529411764706


In [21]:
params = opt.best_params_
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
y_pred = mlp.predict(X_testscaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        60
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         2

    accuracy                           0.88        68
   macro avg       0.29      0.33      0.31        68
weighted avg       0.78      0.88      0.83        68



In [22]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(mlp, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(mlp, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.7666666666666666 ± 0.2905932629027116


1d

In [33]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [34]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [35]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    MLPClassifier(),
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'random_state': [1, 2, 3],
        'solver': ['sgd', 'adam']
    },
    n_iter=32,
    cv=3
)

opt.fit(X_trainscaled, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_testscaled, y_test))

val. score: 0.9013666072489602
test score: 0.8823529411764706


In [36]:
params = opt.best_params_
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
y_pred = mlp.predict(X_testscaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        60
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         2

    accuracy                           0.88        68
   macro avg       0.29      0.33      0.31        68
weighted avg       0.78      0.88      0.83        68



In [37]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(mlp, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(mlp, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.7666666666666666 ± 0.2905932629027116


**Binary**

All features

In [23]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [24]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [25]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    MLPClassifier(),
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'random_state': [1, 2, 3],
        'solver': ['sgd', 'adam']
    },
    n_iter=32,
    cv=3
)

opt.fit(X_trainscaled, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_testscaled, y_test))

val. score: 0.9016636957813429
test score: 0.8970588235294118


In [26]:
params = opt.best_params_
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
y_pred = mlp.predict(X_testscaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93        60
           1       0.40      0.25      0.31         8

    accuracy                           0.87        68
   macro avg       0.65      0.60      0.62        68
weighted avg       0.85      0.87      0.85        68



In [27]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(mlp, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(mlp, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.8 ± 0.24494897427831783


1d

In [28]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [29]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

In [30]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    MLPClassifier(),
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'random_state': [1, 2, 3],
        'solver': ['sgd', 'adam']
    },
    n_iter=32,
    cv=3
)

opt.fit(X_trainscaled, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_testscaled, y_test))

val. score: 0.9013666072489602
test score: 0.8970588235294118


In [31]:
params = opt.best_params_
mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
y_pred = mlp.predict(X_testscaled)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93        60
           1       0.40      0.25      0.31         8

    accuracy                           0.87        68
   macro avg       0.65      0.60      0.62        68
weighted avg       0.85      0.87      0.85        68



In [32]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(mlp, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(mlp, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.8 ± 0.24494897427831783
